In [1]:
import time
from lxml import html
import pandas as pd


### 1. Сбор данных

**Вначале пытался набрать данных на irecommend.ru - оказалось, что ребята не очень хорошо относятся к подобным "сборщикам" и банят легко и непринужденно)) После серии бесплодных попыток решил пока отложить эту затею и заняться чем-нибудь попроще. На слаке набрел на сайт http://deal64.ga, рекомендованный одним из собратьев по специализации. С него данные удалось собрать без особых проблем.**

In [2]:
def get_reviews_links(url):  
    divs = html.parse(url)
    links = divs.xpath("//div[@class='entry']/div[@class='text']/a/@href")
    for l in links:
        yield "http://deal64.ga"+l +"reviews/"
        
def get_start_pages(start_url):
    try:
        divs = html.parse(start_url)
    except:
        return
    num_pages = len(divs.xpath("//div[@class='paging']/a")) + 1
    for i in range(num_pages):
        if (i > 0):
            yield start_url + "page" + str(i) + "/"
        else:
            yield start_url
            
def parse_page(url):
    page= html.parse(url)
    reviews = page.xpath("//div[@class='reviews']/div[@class='entry']")
    res = []
    for r in reviews:
        plus, minus, review = "", "", ""
        
        rate = r.xpath("./div[@class='text'][1]/span[@class='rate']/text()")[0]
        plus = r.xpath("./div[@class='text'][2]/text()")[0]
        minus = r.xpath("./div[@class='text'][3]/text()")[0]
        try: 
            review = r.xpath("./div[@class='text'][4]/text()")[0]        
        except:
            continue
        res.append([plus, minus, review, rate])    
    return res   
            

**Решил поставить задержку хотя бы 0.1 секунду, на всякий случай. Случайно поставил на нолик больше, но тоже прокатило, не забанили))**

In [3]:
%%time
res = []
for start in get_start_pages("http://deal64.ga/mobile/"):
    for s in get_reviews_links(start):
        for rev in get_start_pages(s):
            time.sleep(0.01)
            res = res + parse_page(rev)

Wall time: 5min 14s


In [4]:
data = pd.DataFrame(data=res, columns = ["plus", "minus", "review", "rate"])

In [5]:
data

,plus,minus,review,rate
0,"Недорогой, не скрипит, не люфтит даже через н...",мало места для смс(время от времени приходитс...,"У меня проблем с ним нет, у кого есть - это бр...",5
1,хороший во всём,для FLY аккумуляторных батарей в продаже не н...,"для FLY аккумуляторных батарей нету в продаже,...",5
2,"Низкая цена - 590 рублей! 2 симки, емкий акку...",За такую цену их просто нет!,"Кто пишет, что тихий звук, настройте громкость...",4
3,"цена,динамик и фонарик",отсутствие камеры,"Очень неплохая звонилка,с хорошим качеством зв...",5
4,"Низкая цена, фонарик, крепкий корпус, хорошая...","Плохой звук, мало внутренней памяти для смс и...",Пользуюсь телефоном больше полу года. Нужен бы...,1
5,Дешёвка! Покупался на плюшки от использования...,"Нет ММС, ЖПРС, но и не надо! Не для того эта ...",Телефон неожиданно очень порадовал! 5 с бескон...,5
6,"Цена, 2 сим, маленький, лёгкий, громкий, фона...","Нет стандартного выхода на наушники, кнопка ""...",Покупал 8 месяцев назад для работы. Работаю на...,4
7,"Легкий, шероховатая поверхность (не выронишь)...",не выявил,"Отличный телефон, очень доволен. Советую.",5
8,"Фонарик, удобное меню, низкая цена на данный ...",Ужасный динамик - при разговоре плохо слышно ...,"За свою стоимость может и потянет, но на мой в...",2
9,Цена и 2 симки,"очень слабый динамик, очень плохо слышно собе...","Телефон разочаровал своим динамиком, т.к. оч п...",1


** Итого, получилось собрать результаты с 16870 отзывов в формате: (плюсы, минусы, отзыв, оценка). Сохраним для дальнейших манипуляций**

In [6]:
data.to_csv("deal64.csv", index=False,encoding="utf-8")

### 2. Строим модель

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
import codecs
import re
import pandas as pd

In [8]:
data_train = pd.read_csv("deal64.csv")

In [9]:
with codecs.open("test.csv", encoding="utf-8") as f:
    tests = f.read()
pattern = re.compile("<review>(.+?)</review>", re.MULTILINE|re.DOTALL)  
data_test = pattern.findall(tests)

In [10]:
def make_submission(clf, sub_file_name):
    u"""Создает файл с submission для этого соревнования
    
    clf : обученный классификатор
    sub_file_name : имя выходного файла
    """ 
    d_res = pd.DataFrame()
    pred = clf.predict(data_test)
    d_res["Id"] = range(len(pred))
    d_res["y"] = ["pos" if v == 1 else "neg" for v in pred]
    d_res.to_csv(sub_file_name, index = False)

**Изначально хотел набрать отзывы также из "плюсов" и "минусов", но полистав, решил отказаться от этой идеи - многое перепутано или же просто неинформативно и невозможно классифицировать. Поэтому взял для модели только сами отзывы. Для отклика - посчитал положительными только отзывы с оценкой больше 3:**

In [11]:
texts = data_train.review.values
labels = [1 if v > 3 else 0 for v in data_train.rate.astype(int)]

In [12]:
float(sum(labels))/float(len(labels))

0.7283935981031416

**Попробуем применить простую модель TfidfVectorizer + LogisticRegression. Доля положительных отзывов на обучающей выборке получилась несколько больше, добавим class_weight="balanced" для балансировки**

In [13]:
%%time
pip = Pipeline([
                ("vectorizer", TfidfVectorizer(ngram_range=(1,2))),
                ("classifier", LogisticRegression(class_weight="balanced"))
    ])

score = cross_val_score(pip, texts, labels, cv = 5)
print score
print score.mean()

[ 0.80355556  0.8402963   0.8601067   0.84524162  0.85087459]
0.84001495343
Wall time: 37.2 s


In [14]:
pip.fit(texts, labels)
joblib.dump(pip, "tfid12_logregbalanced.pkl")

['tfid12_logregbalanced.pkl']

In [15]:
make_submission(pip, 'tfid12_logregbalanced.csv' )

**С первого раза на тесте получилась точность 0.96000 - думаю, пока хватит))**